# 主流视频编码压缩技术算法分析

## 一、MPEG-1技术介绍

1. MPEG-1标准发布于1992年，主要应用于VCD、MP3音乐等。
2. 使用MPEG-1的压缩算法可将一部120分钟的电影压缩到1.2GB左右大小，因此，它被广泛应用于VCD制作中。
3. MPEG-1采用块方式运动补偿、离散余弦变换、量化等技术。
4. 是Video CD的核心编码技术。
5. 在空间方向上

- 在图像压缩采用JPEG压缩算法去掉画面内部的冗余信息，即基于DCT的压缩技术，减少空间域冗余。
    
6. 在时间方向上

   采用基于16X16子块的运动补偿。
    
    
### 1、 MPEG-1的层次及语法结构

 ![](./img/data_mpeg_1.png)

1. MPEG-1的数据结构分为6层，从上到下依次是：
 
 
  ![](./img/data_mpeg_1_sgpsmb.png)
####  ①、运动补偿序列(Sequence)

    有表头+图片组+结束标志
 
#### ②、图片组（GOP）

    由一系列图片构成。
 
#### ③、 图片（Picture）

    是基本编码单元，包含一个亮度信号和两个色度信号。
 
#### ④、 图片切片（Slice）

    由一个或多个连续的宏块构成。

#### ⑤、 宏块（MB）

    运动补偿的基本单元。

 
 宏块 ---- 运动补偿的基本单元
 
 块 ----DCT操作的基本单元
 
 
 
### 2、 MPEG的图片组（GOP）

####  ①、为了在高效编码压缩的情况下， 获得可随机存储的高压缩比、高质量图像、MPEG定义了I、P、B三种帧类型。
   
   I： 帧内图（Intra Pictrue）
   P： 预测图（Predicted Picture）
   B： 双向图 (Bidirectional Pictrue)
   
I,P,B 表示1/25s时间间隔的帧序列画面。

MPEG的GOP结构图 （如下图所示）

![](./img/data_mpeg_gop.png)

#####  I帧（帧内图）

I帧采用类似JPEG的编码方式实现，它不以任何其他帧做参考，仅仅进行帧内的空域冗余压缩。

I帧的编码过程简单

 1. 图像进行彩色空间变换，从RGB到YCrCb
 2. 进行区块分割
 3. 对每个区块进行DCT变换
 4. 对DC系数进行差分脉冲编码
 5. 对AC系数进行Zig-Zig的形状排序，然后进行行程编码(RLE)
 6. 最后用Huffman编码。
 
 
数据量大

 由于I帧图像是不参考其他图像帧,只利用本帧的信息进行编码(即无运动预测,采用自身相关性), 因此数据量大。

随机存取

  由于图像序列间无相关性,因此可随机进入图像序列进行编码。
  
I帧编码原理

![](./img/data_mpeg_I_encode_frame.png)
 
##### P帧（预测帧）

 P帧是由一个过去的I帧或P帧采用运动补偿的帧间预测进行更有效编码的方法。
 
 预测图像P使用两种类型的参数来表示：

- 一种参数是当前要编码的图像宏块与参考图像的宏块之间的差值
- 一种是宏块的运动矢量
   
P帧的特点是其本身是前I帧或P帧的前向预测结果,也是产生下一个P帧的基准参考图像。

- 优点：可提供更大的压缩比
- 缺点：前一P帧是下一P帧补偿预测的基准, 如果前者存在误码, 则后者会将编码误差积累起来，传播下去。

P帧的压缩算法原理

![](./img/data_mpeg_frame_p_encode.png)

##### B帧（双向图帧）

- B帧空域提供最高的压缩比, 它即可以用过去的图像帧(I帧或者P帧),也可以用后来的图像(I帧或者P帧)进行运动补偿的双向预测编码方式。
- 由于B帧空域参考下一帧的信息进行编码, 从而减少B帧的大小, 相对P帧更小。
- B帧是同时以前面的I帧或P帧和后面的P帧或I帧为基准进行运动补偿所产生的图像,即双向预测编码。
- 以前面的I帧或P帧代表"过去的信息",  后面的P帧或I帧代表"未来的信息".
- 由于同时使用"过去" 和 "未来"信息, 故称为双向预测帧。

- B帧的压缩算法原理

![](./img/data_mpeg_frame_b_encode.png)


 
